In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import collections
import time
import copy
import math
import numpy as np
from numpy.linalg import matrix_power
from scipy import sparse
import gurobipy as gp
from gurobipy import GRB
from scipy import sparse
import pandas as pd

In [2]:
###################______________________Loading and Preprocessing of USA Dataset______________________###################

node_USA = 221
edge_USA = 2166

w_matrix_USA = sparse.load_npz('usa_mtx_w_n={}_e={}.npz'.format(node_USA, edge_USA))
USA_city_info = pd.read_csv('usa_city_info_n={}_e={}.csv'.format(node_USA, edge_USA))

G_USA = nx.from_scipy_sparse_matrix(w_matrix_USA, create_using=nx.DiGraph, edge_attribute = 'weight')

alpha = np.array(USA_city_info['pop'], dtype=np.float) / 1000000
alpha_USA = {i: alpha[i] for i in range(len(alpha))}
theta_USA = alpha_USA

<ipython-input-2-8a0e3ef931f6>:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  alpha = np.array(USA_city_info['pop'], dtype=np.float) / 1000000


In [3]:
###################______________________Loading and Preprocessing of Facebook Dataset______________________###################

adj_FB = sparse.load_npz('fb_mtx_a_n=600_e=4638.npz')
G_f = nx.from_scipy_sparse_matrix(adj_FB, parallel_edges=False, create_using=nx.Graph, edge_attribute = 'weight')

alpha_f = {node[1]: 1 for node in enumerate(G_f.nodes)}
theta_f = alpha_f

In [4]:
###################______________________Loading and Preprocessing of Twitter Dataset______________________###################

adj_t = sparse.load_npz('twit_mtx_a_n=1000_e=13476.npz')

G_t = nx.from_scipy_sparse_matrix(adj_t, create_using=nx.DiGraph, edge_attribute = 'weight')

alpha_t = {node[1]: 1 for node in enumerate(G_t.nodes)}
theta_t = alpha_t

In [5]:
n1 = 200
n2 = 400
m2 = 4
n3 = 700
m3 = 3


In [6]:
#######################...............Preparing synthetic networks...............#######################

G1 = nx.fast_gnp_random_graph(n1, 0.04, seed=20200905)  # RAND
G2 = nx.powerlaw_cluster_graph(n2, m2, 0.5, seed=20200904)  # POW-S
G3 = nx.powerlaw_cluster_graph(n3, m3, 0.5, seed=20200904)  # POW-L


In [7]:
print(len(G1.edges))
print(len(G2.edges))
print(len(G3.edges))
print(len(G_f.edges()))
print(len(G_t.edges))
print(len(G_USA.edges()))


803
1579
2087
4638
9342
2166


In [8]:
print(len(G1.nodes))
print(len(G2.nodes))
print(len(G3.nodes))
print(len(G_f.nodes))
print(len(G_t.nodes))
print(len(G_USA.nodes))

200
400
700
600
1000
221


In [9]:
alpha1 = np.random.randint(1, 10, size=n1)
weights1 = np.random.uniform(0.3, 1, size=len(G1.edges()))

alpha2 = np.random.randint(1, 10, size=n2)
weights2 = np.random.uniform(0.3, 1, size=len(G2.edges()))

alpha3 = np.random.randint(1, 10, size=n3)
weights3 = np.random.uniform(0.3, 1, size=len(G3.edges()))

alpha1 = {node[1]: alpha1[node[1]] for node in enumerate(G1.nodes)}
theta1 = alpha1

alpha2 = {node[1]: alpha2[node[1]] for node in enumerate(G2.nodes)}
theta2 = alpha2

alpha3 = {node[1]: alpha3[node[1]] for node in enumerate(G3.nodes)}
theta3 = alpha3


In [10]:
nx.set_node_attributes(G1, {node: (theta1[i], alpha1[i]) for i, node in enumerate(G1.nodes)}, 'Attribute')
nx.set_node_attributes(G2, {node: (theta2[i], alpha2[i]) for i, node in enumerate(G2.nodes)}, 'Attribute')
nx.set_node_attributes(G3, {node: (theta3[i], alpha3[i]) for i, node in enumerate(G3.nodes)}, 'Attribute')


In [11]:
nx.set_edge_attributes(G1, values={edge: weights1[i] for i, edge in enumerate(G1.edges)}, name='weight')
nx.set_edge_attributes(G2, values={edge: weights2[i] for i, edge in enumerate(G2.edges)}, name='weight')
nx.set_edge_attributes(G3, values={edge: weights3[i] for i, edge in enumerate(G3.edges)}, name='weight')


In [12]:
%pip install gurobipy


Note: you may need to restart the kernel to use updated packages.


In [13]:
from gurobipy import *


In [14]:
K = 2


In [15]:
V1 = [node[1] for node in enumerate(G1.nodes)]
V2 = [node[1] for node in enumerate(G2.nodes)]
V3 = [node[1] for node in enumerate(G3.nodes)]
V_f = [node[1] for node in enumerate(G_f.nodes)]
V_t = [node[1] for node in enumerate(G_t.nodes)]
V_USA = [node[1] for node in enumerate(G_USA.nodes)]


In [16]:
w_1 = nx.get_edge_attributes(G1, "weight")
w1 = {}
for y in w_1:
    w1[y[1], y[0]] = w_1[y[0], y[1]]
    w1[y[0], y[1]] = w_1[y[0], y[1]]


w_2 = nx.get_edge_attributes(G2, "weight")
w2 = {}
for y in w_2:
    w2[y[1], y[0]] = w_2[y[0], y[1]]
    w2[y[0], y[1]] = w_2[y[0], y[1]]

w_3 = nx.get_edge_attributes(G3, "weight")
w3 = {}
for y in w_3:
    w3[y[1], y[0]] = w_3[y[0], y[1]]
    w3[y[0], y[1]] = w_3[y[0], y[1]]

wf = nx.get_edge_attributes(G_f, "weight")
w_f = {}
for y in wf:
    w_f[y[1], y[0]] = wf[y[0], y[1]]
    w_f[y[0], y[1]] = wf[y[0], y[1]]

wt = nx.get_edge_attributes(G_t, "weight")
w_t = {}
for y in wt:
    w_t[y[1], y[0]] = wt[y[0], y[1]]
    w_t[y[0], y[1]] = 0

w_usa = nx.get_edge_attributes(G_USA, "weight")
w_USA = {}
for y in w_usa:
    w_USA[y[1], y[0]] = w_usa[y[0], y[1]]
    w_USA[y[0], y[1]] = 0


In [17]:
def milp_model(G, V, theta, w, alpha):
    R_sol = []
    model = Model("MILP_Model")

    t_keys = []
    x_keys = []
    r_keys = []
    k_nbrs = {}
    
   
    for u in V:
        r_keys.append(u)
        k_nbrs[u] = []
        for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
            k_nbrs[u].append(v)
            x_keys.append((u, v))
            for z in nx.single_source_shortest_path_length(G, v, cutoff=1).keys():
                if(z != v):
                    t_keys.append((u, v, z))

    # Decision variables
    P_Loss = model.addVar(vtype=GRB.CONTINUOUS, name="P_Loss")
    R = model.addVar(vtype=GRB.CONTINUOUS, name="R")               
    x = {key: 1 for key in x_keys}
    r = model.addVars(r_keys, vtype=GRB.CONTINUOUS, name='r')
    t = model.addVars(t_keys, vtype=GRB.CONTINUOUS, name='b')

    model.addConstr(sum(r.values()) <= R, name='const1')
    model.addConstrs((t[(i, j, k)] <= w[j, k] * r[j]
                     for (i, j, k) in t_keys), name='borrowing limitations')
    
    model.addConstrs((gp.quicksum((1 - x[(i, k)]) * alpha[k] for k in k_nbrs[i]) <= P_Loss
                      for i in r_keys), name='minimize maximum')
    for ra in r_keys:
        model.addConstrs((r[k] - gp.quicksum(t.select(ra, k, '*')) + gp.quicksum(t.select(ra, '*', k))
                          >= theta[k] * x[(ra, k)] for k in k_nbrs[ra]), name='reallocation results for node {}'.format(ra))
        model.addConstrs((gp.quicksum(t.select(ra, k, '*')) <= r[k] for k in r_keys),
                         name='total out-borrowings')
    
    # Objective function
    model.setObjective(R, GRB.MINIMIZE)

    model.optimize()
    
    R_sol.append(model.getObjective().getValue())
    
    return max(R_sol)


In [18]:
R_sol_1 = milp_model(G1, V1, theta1, w1, alpha1)
R_sol_2 = milp_model(G2, V2, theta2, w2, alpha2)
R_sol_3 = milp_model(G3, V3, theta3, w3, alpha3)
R_sol_f = milp_model(G_f, V_f, theta_f, w_f, alpha_f)
R_sol_t = milp_model(G_t, V_t, theta_t, w_t, alpha_t)
R_sol_USA = milp_model(G_USA, V_USA, theta_USA, w_USA, alpha_USA)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-13
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 160336 rows, 107965 columns and 535357 nonzeros
Model fingerprint: 0x4362c9d8
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 84060 rows and 56233 columns
Presolve time: 0.19s
Presolved: 76276 rows, 51732 columns, 282404 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.09s

Barrier statistics:
 Dense cols : 190
 AA' NZ     : 3.671e+05
 Factor NZ  : 1.715e+06 (roughly 70 MB of memory)
 Factor Ops : 1.442e+08 (less than 1 second per iteration)
 Threads    : 3

     

In [ ]:
print(R_sol_1, R_sol_2, R_sol_3, R_sol_USA, R_sol_f, R_sol_t)

NameError: name 'R_sol_USA' is not defined

In [ ]:
w_1 = nx.get_edge_attributes(G1, "weight")
w1_zero = {}
for y in w_1:
    w1_zero[y[1], y[0]] = 0
    w1_zero[y[0], y[1]] = 0


w_2 = nx.get_edge_attributes(G2, "weight")
w2_zero = {}
for y in w_2:
    w2_zero[y[1], y[0]] = 0
    w2_zero[y[0], y[1]] = 0

w_3 = nx.get_edge_attributes(G3, "weight")
w3_zero = {}
for y in w_3:
    w3_zero[y[1], y[0]] = 0
    w3_zero[y[0], y[1]] = 0

wf = nx.get_edge_attributes(G_f, "weight")
w_f_zero = {}
for y in wf:
    w_f_zero[y[1], y[0]] = 0
    w_f_zero[y[0], y[1]] = 0

wt = nx.get_edge_attributes(G_t, "weight")
w_t_zero = {}
for y in wt:
    w_t_zero[y[0], y[1]] = 0
    w_t_zero[y[1], y[0]] = 0

w_usa = nx.get_edge_attributes(G_USA, "weight")
w_USA_zero = {}
for y in w_usa:
    w_USA_zero[y[0], y[1]] = 0
    w_USA_zero[y[1], y[0]] = 0


In [ ]:
R_sol_wo_realloc_1 = milp_model(G1, V1, theta1, w1_zero, alpha1)
R_sol_wo_realloc_2 = milp_model(G2, V2, theta2, w2_zero, alpha2)
R_sol_wo_realloc_3 = milp_model(G3, V3, theta3, w3_zero, alpha3)
R_sol_wo_realloc_f = milp_model(G_f, V_f, theta_f, w_f_zero, alpha_f)
R_sol_wo_realloc_t = milp_model(G_t, V_t, theta_t, w_t_zero, alpha_t)
R_sol_wo_realloc_USA = milp_model(G_USA, V_USA, theta_USA, w_USA_zero, alpha_USA)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 160336 rows, 107965 columns and 427594 nonzeros
Model fingerprint: 0xdb83620a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 160336 rows and 107965 columns
Presolve time: 0.09s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.5300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.14 seconds (0.09 work units)
Optimal objective  9.530000000e+02
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physi

In [ ]:
print(R_sol_wo_realloc_1, R_sol_wo_realloc_2, R_sol_wo_realloc_3, R_sol_wo_realloc_USA, R_sol_wo_realloc_f, R_sol_wo_realloc_t)

953.0 1931.0 3413.0 340.619965 600.0 1000.0


In [ ]:
def Greedy(G, V, theta, alpha):
    R = 0.5*sum(theta.values())
    max_loss = 0
    #alpha_x = {v:alpha[v] for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys()}
    sorted_alpha = dict(
        sorted(alpha.items(), key=lambda x: x[1], reverse=True))
    allocation = {key: 0 for key in sorted_alpha.keys()}
    allocated = 0

    for key in sorted_alpha.keys():
        if((R-allocated) >= theta[key]):
            allocation[key] = theta[key]
            allocated += allocation[key]
        else:
            allocation[key] = R-allocated
            break

    for u in V:

        alpha_x = {v: alpha[v] for v in nx.single_source_shortest_path_length(
            G, u, cutoff=K).keys()}
        #sorted_alpha = dict(sorted(alpha_x.items(), key=lambda x:x[1], reverse=True))
        #allocation={key:0 for key in sorted_alpha.keys()}
        # allocated=0

        loss = 0
        for key in alpha_x.keys():
            if(allocation[key] < theta[key]):
                loss += alpha_x[key]

        if(loss > max_loss):
            max_loss = loss

    return max_loss, allocation


In [ ]:
loss_greedy1, allocation1 = Greedy(G1, V1, theta1, alpha1)
loss_greedy2, allocation2 = Greedy(G2, V2, theta2, alpha2)
loss_greedy3, allocation3 = Greedy(G3, V3, theta3, alpha3)
loss_greedy_USA, allocation_USA = Greedy(G_USA, V_USA, theta_USA, alpha_USA)
loss_greedy_f, allocation_f = Greedy(G_f, V_f, theta_f, alpha_f)
loss_greedy_t, allocation_t = Greedy(G_t, V_t, theta_t, alpha_t)


In [ ]:
print(loss_greedy1, loss_greedy2, loss_greedy3, loss_greedy_USA , loss_greedy_f, loss_greedy_t)


296 855 1170 178.68352000000007 188 301


In [ ]:
def Reallocation(G, V):
    reallocation = {}
    for u in V:
        for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
            for w in nx.single_source_shortest_path_length(G, v, cutoff=1).keys():
                reallocation[(u, v, w)] = 0
                reallocation[(u, w, v)] = 0
    return reallocation


In [ ]:
reallocation1 = Reallocation(G1, V1)
reallocation2 = Reallocation(G2, V2)
reallocation3 = Reallocation(G3, V3)
reallocation_f = Reallocation(G_f, V_f)
reallocation_t = Reallocation(G_t, V_t)
reallocation_USA = Reallocation(G_USA, V_USA)


In [ ]:
def reallocation_from_neighbour(G, u, target, reallocation):
    totalReallocation = 0.0
    for vertex in nx.single_source_shortest_path_length(G, target, cutoff=1).keys():
        totalReallocation += reallocation[(u, vertex, target)]
    return totalReallocation


In [ ]:
def Greedy_R(G, V, theta, w, alpha, Allocation, reallocation):
    max_loss = 0
    allocation = Allocation
    defending_power = {(u, v): Allocation[v] for u in V for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys()}
    for u in V:
        alpha_x = {v: alpha[v] for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys() if Allocation[v] < theta[v]}
        sorted_alpha_x = dict(sorted(alpha_x.items(), key=lambda x: x[1], reverse=True))
        for key in sorted_alpha_x.keys():
            for neighbour in nx.single_source_shortest_path_length(G, key, cutoff=1).keys():
                if neighbour==key:
                    continue
                if neighbour not in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
                    curr_demand = theta[key] - Allocation[key] - reallocation_from_neighbour(G, u, key, reallocation)
                    if(curr_demand > 0):
                        if(w[neighbour, key]*allocation[neighbour] <= curr_demand):
                            reallocation[(u, neighbour, key)] += w[neighbour, key]*allocation[neighbour]
                            allocation[neighbour] = 0
                        else:
                            reallocation[(u, neighbour, key)] += curr_demand
                            allocation[neighbour] -= curr_demand
                    else:
                        break
    # print(reallocation)

    for u in V:
        for vertex in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
            for neighbour in nx.single_source_shortest_path_length(G, vertex, cutoff=1).keys():
                if(neighbour!=vertex):
                    if neighbour in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
                        defending_power[(u, vertex)] += (reallocation[(u, neighbour,vertex)] - reallocation[(u, vertex, neighbour)])
                    else:
                        defending_power[(u, vertex)] += reallocation[(u, neighbour, vertex)]

    for u in V:
        loss = 0
        for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
            if(defending_power[(u, v)] < theta[v]):
                loss += alpha[v]
        if(loss > max_loss):
            max_loss = loss

    return max_loss


In [ ]:
loss_greedy_r1 = Greedy_R(G1, V1, theta1, w1, alpha1, allocation1, reallocation1)
loss_greedy_r2 = Greedy_R(G2, V2, theta2, w2, alpha2, allocation2, reallocation2)
loss_greedy_r3 = Greedy_R(G3, V3, theta3, w3, alpha3, allocation3, reallocation3)
loss_greedy_r_USA = Greedy_R(G_USA, V_USA, theta_USA, w_USA, alpha_USA, allocation_USA, reallocation_USA)
loss_greedy_r_f = Greedy_R(G_f, V_f, theta_f, w_f, alpha_f, allocation_f, reallocation_f)
loss_greedy_r_t = Greedy_R(G_t, V_t, theta_t, w_t, alpha_t, allocation_t, reallocation_t)


In [ ]:
print(loss_greedy_r1, loss_greedy_r2, loss_greedy_r3, loss_greedy_r_USA, loss_greedy_r_f, loss_greedy_r_t)

296 855 1169 178.68352000000007 188 301


In [ ]:
#######################...............Defining the LP Model for BA_epsilon algorithm...............#######################

def BA_epsilon(G, V, theta, w, alpha, epsilon):
    model = Model("LP_Model")

    R = epsilon*0.5*sum(theta.values())
    
    t_keys = []
    x_keys = []
    r_keys = []
    k_nbrs = {}
    
   
    for u in V:
        r_keys.append(u)
        k_nbrs[u] = []
        for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
            k_nbrs[u].append(v)
            x_keys.append((u, v))
            for z in nx.single_source_shortest_path_length(G, v, cutoff=1).keys():
                if(z != v):
                    t_keys.append((u, v, z))

    # Decision variables
    P_Loss = model.addVar(vtype=GRB.CONTINUOUS, name="P_Loss")               
    x = model.addVars(x_keys, lb=0, ub=1, vtype=GRB.CONTINUOUS, name='x')
    r = model.addVars(r_keys, vtype=GRB.CONTINUOUS, name='r')
    t = model.addVars(t_keys, vtype=GRB.CONTINUOUS, name='b')

    model.addConstr(sum(r.values()) <= R, name='const1')
    model.addConstrs((t[(i, j, k)] <= w[j, k] * r[j]
                     for (i, j, k) in t_keys), name='borrowing limitations')
    
    model.addConstrs((gp.quicksum((1 - x[(i, k)]) * alpha[k] for k in k_nbrs[i]) <= P_Loss
                      for i in r_keys), name='minimize maximum')
    for ra in r_keys:
        model.addConstrs((r[k] - gp.quicksum(t.select(ra, k, '*')) + gp.quicksum(t.select(ra, '*', k))
                          >= theta[k] * x[(ra, k)] for k in k_nbrs[ra]), name='reallocation results for node {}'.format(ra))
        model.addConstrs((gp.quicksum(t.select(ra, k, '*')) <= r[k] for k in r_keys),
                         name='total out-borrowings')
    
    # Objective function
    model.setObjective(P_Loss, GRB.MINIMIZE)

    model.optimize()

    
    if(model.status != GRB.OPTIMAL):
        return None
    else:
        solution_x = {}
        sol_x = {}
        for u in V:
            for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
                sol_x[u, v] = x[u, v].X
                if(x[u, v].X > epsilon):
                    solution_x[u, v] = 1
                else:
                    solution_x[u, v] = 0

        max_loss = 0
        for u in V:
            loss = 0
            for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
                loss += (1-solution_x[u, v])*alpha[v]
            if(loss > max_loss):
                max_loss = loss

        return max_loss, sol_x, model.getObjective().getValue()


In [ ]:
#######################...............Defining the LP Model for BA_tau algorithm...............#######################

def BA_tau(G, V, theta, w, alpha, solu_x):
    model = Model("LP_Model")

    R = 0.5*sum(theta.values())
    
    t_keys = []
    x_keys = []
    r_keys = []
    k_nbrs = {}
    sol_x = solu_x
   
    for u in V:
        r_keys.append(u)
        k_nbrs[u] = []
        for v in nx.single_source_shortest_path_length(G, u, cutoff=K).keys():
            k_nbrs[u].append(v)
            x_keys.append((u, v))
            for z in nx.single_source_shortest_path_length(G, v, cutoff=1).keys():
                if(z != v):
                    t_keys.append((u, v, z))

    # Decision variables
    P_Loss = model.addVar(vtype=GRB.CONTINUOUS, name="P_Loss")               
    x = model.addVars(x_keys, lb=0, ub=1, vtype=GRB.CONTINUOUS, name='x')
    r = model.addVars(r_keys, vtype=GRB.CONTINUOUS, name='r')
    t = model.addVars(t_keys, vtype=GRB.CONTINUOUS, name='b')


    model.addConstr(sum(r.values()) <= R, name='const1')
    model.addConstrs((t[(i, j, k)] <= w[j, k] * r[j]
                     for (i, j, k) in t_keys), name='borrowing limitations')
    
    model.addConstrs((gp.quicksum((1 - x[(i, k)]) * alpha[k] for k in k_nbrs[i]) <= P_Loss
                      for i in r_keys), name='minimize maximum')
    for ra in r_keys:
        model.addConstrs((r[k] - gp.quicksum(t.select(ra, k, '*')) + gp.quicksum(t.select(ra, '*', k))
                          >= theta[k] * x[(ra, k)] for k in k_nbrs[ra]), name='reallocation results for node {}'.format(ra))
        model.addConstrs((gp.quicksum(t.select(ra, k, '*')) <= r[k] for k in r_keys),
                         name='total out-borrowings')
    
    # Objective function
    model.setObjective(P_Loss, GRB.MINIMIZE)

    flag=0
    for key in sol_x.keys():
        if sol_x[key]<0 and sol_x[key]<1:
            flag=2
            break
    if flag==2:
        tau = 0
        tau_low = 0
        tau_high = 0
    else:
        tau_low = 0
        tau_high = max(sol_x.values())
        flag=3
    
    mark=0
    while(tau_high - tau_low >= 0.05):
        tau = (tau_high + tau_low)/2
        sol_x = solu_x
        for key in sol_x.keys():
            if sol_x[key] > tau:
                sol_x[key] = 1
            else:
                sol_x[key] = 0
        for u in V:
            for v in k_nbrs[u]:
                if sol_x[u, v] == 1:
                    model.addConstr(x[(u,v)] >= sol_x[u, v], name='x_fixed for node {}_{}'.format(u,v))
                else:
                    model.addConstr(x[(u,v)] <= sol_x[u, v], name='x_fixed for node {}_{}'.format(u,v))
        
        model.update()
        model.optimize()
        if model.status != GRB.OPTIMAL:  # Infeasible
            tau_low = tau
            flag = 0
        else:
            tau_high = tau
            flag = 1
            mark = 1  # exist at least one feasible tau
    
    if mark==0:
        flag=3
    
    
    if flag==3:    # if tau_high is too low
        tau = tau_high
        sol_x = solu_x
        for key in sol_x.keys():
            if sol_x[key] > tau:
                sol_x[key] = 1
            else:
                sol_x[key] = 0
        for u in V:
            for v in k_nbrs[u]:
                obj_constrain = model.getConstrByName('x_fixed for node {}_{}'.format(u,v))
                model.remove(obj_constrain)
        for u in V:
            for v in k_nbrs[u]:
                if sol_x[u, v] == 1:
                    model.addConstr(x[(u,v)] >= sol_x[u, v], name='x_fixed for node {}_{}'.format(u,v))
                else:
                    model.addConstr(x[(u,v)] <= sol_x[u, v], name='x_fixed for node {}_{}'.format(u,v))
        model.update()
        model.optimize()
    
    if flag == 0:
        tau = tau_high
        sol_x = solu_x
        for key in sol_x.keys():
            if sol_x[key] > tau:
                sol_x[key] = 1
            else:
                sol_x[key] = 0
    
    max_loss = 0
    for u in V:
        loss = 0
        for v in k_nbrs[u]:
            loss += (1-sol_x[u, v])*alpha[v]
        if(loss > max_loss):
            max_loss = loss

    return tau, max_loss, 


In [ ]:
def BA(G, V, theta, w, alpha):
    loss_epsilon = {}
    loss_epsilon_tau = {}
    defend_result_LP = {}
    epsilons = [0.9,1]
    for epsilon in epsilons:
        loss_epsilon[epsilon], solu_x, defend_result_LP[epsilon] = BA_epsilon(G, V, theta, w, alpha, epsilon)
        tau, loss_epsilon_tau[epsilon, tau] = BA_tau(G, V, theta, w, alpha, solu_x)
    return loss_epsilon, loss_epsilon_tau, defend_result_LP


In [ ]:
BA_epsilon_loss1, BA_tau_loss1, exact_defend_result1 = BA(G1, V1, theta1, w1, alpha1)
BA_epsilon_loss2, BA_tau_loss2, exact_defend_result2 = BA(G2, V2, theta2, w2, alpha2)
BA_epsilon_loss3, BA_tau_loss3, exact_defend_result3 = BA(G3, V3, theta3, w3, alpha3)
BA_epsilon_loss_USA, BA_tau_loss_USA, exact_defend_result_USA = BA(G_USA, V_USA, theta_USA, w_USA, alpha_USA)
BA_epsilon_loss_f, BA_tau_loss_f, exact_defend_result_f = BA(G_f, V_f, theta_f, w_f, alpha_f)
BA_epsilon_loss_t, BA_tau_loss_t, exact_defend_result_t = BA(G_t, V_t, theta_t, w_t, alpha_t)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 160336 rows, 120336 columns and 560100 nonzeros
Model fingerprint: 0x4bd1c6b3
Coefficient statistics:
  Matrix range     [3e-01, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 5e+02]
Presolve removed 83859 rows and 56231 columns
Presolve time: 0.34s
Presolved: 76477 rows, 64105 columns, 307548 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.13s

Barrier statistics:
 Dense cols : 191
 AA' NZ     : 3.808e+05
 Factor NZ  : 1.757e+06 (roughly 70 MB of memory)
 Factor Ops : 1.506e+08 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Prim

In [ ]:
print(min(BA_epsilon_loss1.values()), min(BA_tau_loss1.values()), min(exact_defend_result1.values()))
print(min(BA_epsilon_loss2.values()), min(BA_tau_loss2.values()), min(exact_defend_result2.values()))
print(min(BA_epsilon_loss3.values()), min(BA_tau_loss3.values()), min(exact_defend_result3.values()))
print(min(BA_epsilon_loss_USA.values()), min(BA_tau_loss_USA.values()), min(exact_defend_result_USA.values()))
print(min(BA_epsilon_loss_f.values()), min(BA_tau_loss_f.values()), min(exact_defend_result_f.values()))
print(min(BA_epsilon_loss_t.values()), min(BA_tau_loss_t.values()), min(exact_defend_result_t.values()))

257 197 152.68940520798503
989 833 828.4999999999997
926 673 661.0000000000002
287.44357100000013 228.63410600000003 170.2467684999998
96 60 52.500000000000064
168 168 142.99999999999994


In [ ]:
print(sum(theta1.values()))
print(sum(theta2.values()))
print(sum(theta3.values()))
print(sum(theta_f.values()))
print(sum(theta_t.values()))
print(sum(theta_USA.values()))


999
2059
3630
600
1000
340.61996500000015


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(["POW-S", "POW-L", "RAND", "FB", "Twit"], [min_R_with_reallocation_1, min_R_with_reallocation_2, min_R_with_reallocation_3, min_R_with_reallocation_f,
            min_R_with_reallocation_t, min_R_with_reallocation_USA], c="pink", linewidths=2, marker="s", edgecolor="green", s=50, label="With_Reallocation")
plt.scatter(["POW-S", "POW-L", "RAND", "FB", "Twit"], [min_R_without_reallocation_1, min_R_without_reallocation_2, min_R_without_reallocation_3, min_R_without_reallocation_f,
            min_R_without_reallocation_t, min_R_without_reallocation_USA], c="red", linewidths=2, marker="^", edgecolor="blue", s=50, label="Without_Reallocation")
plt.legend(bbox_to_anchor=(1.05, 0.8), ncol=1, loc="upper left")
plt.ylabel("Minimum Resource Required")
plt.title("Minimum Resource Required for perfect defending strategy")
